In [ ]:
import torch 
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchsummary import summary
from tqdm.notebook import tqdm


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device using: ',device)
if device != 'cpu':
    print(torch.cuda.get_device_name())

In [ ]:
dataset = ImageFolder('../dataset',transform=transforms.Compose([transforms.ToTensor(), transforms.Grayscale(num_output_channels=1)]))
print('Length of the dataset is ',len(dataset))
print('Classes',dataset.classes, '\nNumber of classes: ',len(dataset.classes))

In [ ]:
image,label = dataset[0]
image.shape , label , dataset.classes[label]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt 

def display(data):
    image,label = data 
    plt.imshow(image.permute(1,2,0))
    print(f'Label {dataset.classes[label]} Image shape: {image.shape}')
display(dataset[-187])


In [ ]:
o = lambda i,k,p=0,s=1: (i+2*p-k)//s +1 
o(16,3) , 32/2

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.conv1 = nn.Conv2d(1,6,3) # out 32
        self.pool1 = nn.MaxPool2d(2,2) # out 16
        self.conv2 = nn.Conv2d(6,16,3) # out 14
        self.pool2 = nn.MaxPool2d(2,2) # out 7
        self.conv3 = nn.Conv2d(16,26,3) # out 5
        self.fc1 = nn.Linear(25*26,64)
        self.fc2 = nn.Linear(64,26)
    
    def forward(self,x):
        x = nn.functional.relu(self.conv1(x))
        x = self.pool1(x)
        x = nn.functional.relu(self.conv2(x))
        x = self.pool2(x)
        x = nn.functional.relu(self.conv3(x))
        x = x.view(-1,25*26)
        x = self.fc1(x)
        x = self.fc2(x)
        # x1 = nn.functional.softmax(x)

        return x 



In [ ]:
model = Net().to(device)
summary(model,(1,34,34))

In [ ]:
criterion = nn.CrossEntropyLoss()
learning_rate = 3e-4
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [ ]:
def training_process(model,device,trainloader,valoader,criterion,optimizer):
    model.train()
    running_loss  = 0
    Ncorrect_predictions = 0 # number of correct prediction
    total = 0
    for images,labels in tqdm(trainloader):
        images,labels = images.to(device), labels.to(device)
        # moving the data to GPU
        optimizer.zero_grad()
        out = model(images)
        loss_per_batch = criterion(out,labels)
        total += labels.size(0) # basically this should return the batch size
        preds = torch.argmax(out,1).cpu().detach()
        running_loss +=loss_per_batch.item()
        Ncorrect_predictions += (preds==labels.cpu().detach()).sum().item()
        
        loss_per_batch.backward() # back propagation algorithm
        optimizer.step()

    acc = 100*(Ncorrect_predictions/total)
    avg_loss = running_loss/len(trainloader)

    model.eval()
    total_ = 0
    correct_ = 0
    running_loss_ = 0
    with torch.no_grad():
        for images,labels in tqdm(valoader):
            out = model(images.to(device))
            labels = labels.to(device)
            loss = criterion(out,labels)
            running_loss_ += loss 
            total_ += labels.size(0)
            predictions = out.argmax(dim=1).cpu().detach()
            correct_ += (predictions==labels.cpu().detach()).sum().item()
    
    val_loss = running_loss_/len(valoader)
    val_acc = 100*(correct_/total_)

    return acc,avg_loss,val_loss,val_acc

    

In [ ]:
from PIL import Image 
import numpy as np
x = Image.open('apple0.jpg')
x=x.convert('L')
y = np.asarray(x)
print(y.shape)
# x.show()
x1 = x.resize((34,34))
# x1.show()
x1 = torch.from_numpy(np.asarray(x1))
x1 = x1.unsqueeze(0)
x1 = x1.unsqueeze(0)
print(x1.shape)
x1=x1.float().to(device)
print(type(x1),device)
out = model(x1)
print(dataset.classes[out.argmax(1).item()])
out = nn.functional.softmax(out,1)
print(dataset.classes[out.argmax(1).item()])


In [ ]:
import cv2 
x = cv2.imread('../apple0.jpg')
print(x.shape)
hsv = cv2.cvtColor(x,cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv,np.array([0,0,250]),np.array([172,111,255]))
# _, thresh = cv2.threshold(y,250,255,0)
contours, _ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
cv2.drawContours(x,contours,-1,(0,255,255),3)
cv2.imshow('out',x)
print(contours,a)

In [ ]:
S = len(dataset)
S*0.7,S*0.1,S*0.2,S

In [ ]:

train,val,test = torch.utils.data.random_split(dataset, [4781,684,1366])
len(train), len(val), len(test)
batch = 256
trainDL = DataLoader(train,batch,shuffle=True,num_workers=4)
valDL = DataLoader(val,batch,shuffle=False,num_workers=4)
# testDL = DataLoader(test,batch,shuffle=False,num_workers=4)
len(trainDL),len(valDL)

In [ ]:
train_losses=[]
valid_losses= []
accuracy=[]
v=[]
for i in range(80):
    acc,avg_loss,val_loss,val_acc = training_process(model,device,trainDL,valDL,criterion,optimizer)
    print(f'Epoch{i+1}\nTrain: acc {acc} loss {avg_loss}\nVal: acc{val_acc} loss {val_loss}')
    train_losses.append(avg_loss)
    valid_losses.append(val_loss)
    accuracy.append(acc)
    v.append(val_acc)

In [ ]:
def plot(nb_epochs,train_losses,valid_losses,acc,v):
    epochs = range(nb_epochs)
    plt.plot(epochs, train_losses, 'g', label='Training loss')
    plt.plot(epochs, valid_losses, 'b', label='validation loss')
    plt.title('Training and Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    plt.plot(epochs, acc, 'g', label='Training accuracy')
    plt.plot(epochs, v, 'b', label='validation accuracy')
    plt.title('Training and Validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('accuracy')
    plt.legend()
    plt.show()

In [ ]:

plot(100-20,train_losses,[v.cpu() for v in valid_losses],accuracy,v)

In [ ]:
torch.save(model.state_dict(),'Model.pth')

In [ ]:
def Final_test(dataloader,model,loss_fn):

#     model in evaluation mode
    model.eval() 
    
    total = 0
    correct = 0
    running_loss = 0
    
    with torch.no_grad(): # gradients calculation not required
        
        for x,y in dataloader:
            
            output        = model(x.to(device)) #model's output.
            loss          = loss_fn(output,y.to(device)).item() #loss calculation.
            running_loss += loss
            
            total        += y.size(0)
            predictions   = output.argmax(dim=1).cpu().detach()
            correct      += (predictions == y.cpu().detach()).sum().item()
            
    avg_loss = running_loss/len(dataloader) #Average loss per batch.      
    val_acc = 100*(correct/total)
    print(f'\t test Loss = {avg_loss:.6f}',end='\t')
    print(f'Accuracy on test set = {100*(correct/total):.6f}% [{correct}/{total}]') #Prints the Accuracy.
    


In [ ]:
testDL = DataLoader(test,batch,shuffle=False,num_workers=4)

Final_test(testDL,model,criterion)